In [ ]:
# default_exp filter

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# 01_06_Pivot_BS_Data

In this notebook, we will transform the verticalized data rows of the BalanceSheet into a horizontalized dataframe.
<br>
Currently, our data looks similar to the table below. Every Value is placed on its own row.


| bs_id | company    | date       | attribute | value |
|-------|------------|------------|-----------|-------|
| 1     | VitaSport  | 31.10.2018 | Assets    | 100   |
| 1     | VitaSport  | 31.10.2018 | Cash      | 80    |
| 1     | VitaSport  | 31.10.2018 | Other     | 20    |
| 2     | VitaSport  | 31.10.2019 | Assets    | 120   |
| 2     | VitaSport  | 31.10.2019 | Cash      | 80    |
| 2     | VitaSport  | 31.10.2019 | Other     | 40    |
| 3     | GloryFood  | 31.10.2019 | Assets    | 50    |
| 3     | GloryFood  | 31.10.2019 | Cash      | 5     |
| 3     | GloryFood  | 31.10.2019 | Other     | 45    |

<br>
But what we would like to have one entry per BalanceSheet:

| bs_id | company   | date       | Assets | Cash | Other |
|-------|-----------|------------|--------|------|-------|
| 1     | VitaSport | 31.10.2018 | 100    | 80   | 20    |
| 2     | VitaSport | 31.10.2019 | 120    | 80   | 40    |
| 3     | GloryFood | 31.10.2019 | 50     | 5    | 45    |

In [ ]:
# imports
from bfh_cas_bgd_fs2020_sa.core import * # initialze spark

from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.types import *

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
# folder with our test-dataset which contains only data from two zip files
tst_filtered_folder = "./tmp/filtered/"
tst_bs_folder = "./tmp/bs/"

# folder with the whole dataset as a single parquet
all_filtered_folder = "D:/data/parq_filtered"
all_bs_folder = "D:/data/parq_bs"

## Init Spark

In [ ]:
spark = get_spark_session() # Session anlegen
spark # display the most important information of the session

## Load the dataset

Loading the data doesn't really do anything. It just prepares the df. But we well use the cache() method to keep the data in memory, once it is loaded for the first time.

### Load the test data

In [ ]:
df_tst = spark.read.parquet(tst_filtered_folder).cache()

### Load the whole dataset

In [ ]:
df_all = spark.read.parquet(all_filtered_folder).cache()

### Print all the contained column names

In [ ]:
_ = [print(x, end=", ") for x in df_all.columns] # print the name of the columns for convenience

cik, adsh, tag, version, coreg, ddate, qtrs, uom, value, footnote, name, sic, countryba, stprba, cityba, zipba, bas1, bas2, baph, countryma, stprma, cityma, zipma, mas1, mas2, countryinc, stprinc, ein, former, changed, afs, wksi, fye, form, period, fy, fp, filed, accepted, prevrpt, detail, instance, nciks, aciks, report, line, stmt, inpth, rfile, plabel, negating, ticker, name_cik_tic, exchange, cik_select, 

## Loading data into memory

We just make a count on the test and the all dataset. This ensure that the data will be loaded into the memory and is cached afterwards.

In [ ]:
start = time.time()
print("Entries in Test: ", "{:_}".format(df_tst.count())) # loading test dataset into memory
duration = time.time() - start
print("duration: ", duration)

Entries in Test:  1_680_108
duration:  13.891040802001953


In [ ]:
start = time.time()
print("Entries in Test: ", "{:_}".format(df_all.count())) # loading all dataset into memory
duration = time.time() - start
print("duration: ", duration)

Entries in Test:  35_454_045
duration:  224.06099796295166


Since we filtered out about two thirds of the entries, loading the reduced data set takes only about 3 minutes to load it completely into memory

## Basics

In order to test how to pivot the data, we implement a simple example to test the principle. Actually, der is a pivot function, which provides the desired functionality.

In [ ]:
df_bs_data = spark.createDataFrame( \
[ \
    (1,"VitaSport","31.10.2018","Assets",100), \
    (1,"VitaSport","31.10.2018","Cash  ",80 ), \
    (1,"VitaSport","31.10.2018","Other ",20 ), \
    (2,"VitaSport","31.10.2019","Assets",120), \
    (2,"VitaSport","31.10.2019","Cash  ",80 ), \
    (2,"VitaSport","31.10.2019","Other ",40 ), \
    (3,"GloryFood","31.10.2019","Assets",50 ), \
    (3,"GloryFood","31.10.2019","Cash  ",5  ), \
    (3,"GloryFood","31.10.2019","Other ",45 )  \
], \
  ("bs_id", "company", "date", "attribute", "value") \
)

df_bs_data.groupby(["company","bs_id","date"]).pivot("attribute").max("value").show()

+---------+-----+----------+------+------+------+
|  company|bs_id|      date|Assets|Cash  |Other |
+---------+-----+----------+------+------+------+
|VitaSport|    2|31.10.2019|   120|    80|    40|
|VitaSport|    1|31.10.2018|   100|    80|    20|
|GloryFood|    3|31.10.2019|    50|     5|    45|
+---------+-----+----------+------+------+------+



This looks simple. But it could be, that we will get more than one result. In the above sample, we just used the max aggregate function. However, that might be a too simple solution for real data.

## Pivoting Apple in the Testdata

In a first step, we select only the BalanceSheet data of Apple in the testset and we expect to have 2 BalanceSheets in there (one for every quarter - since the testset contains two quarter of data.

In [ ]:
apple_df = df_tst.where("cik == 320193 and stmt = 'BS'").cache()

Check how many datarows there are for Apple in the two test quarters.

In [ ]:
apple_df.count()

134

In [ ]:
apple_vip_cols = apple_df.select(['cik','adsh','period','tag', 'version', 'ddate','uom','value', 'qtrs','fp', 'report','line'])

In [ ]:
apple_vip_cols.show()

+------+--------------------+----------+--------------------+------------+----------+---+----------+----+---+------+----+
|   cik|                adsh|    period|                 tag|     version|     ddate|uom|     value|qtrs| fp|report|line|
+------+--------------------+----------+--------------------+------------+----------+---+----------+----+---+------+----+
|320193|0000320193-19-000119|2019-09-30|  LiabilitiesCurrent|us-gaap/2019|2018-09-30|USD|1.15929E11|   0| FY|     4|  23|
|320193|0000320193-19-000119|2019-09-30|  LiabilitiesCurrent|us-gaap/2019|2019-09-30|USD|1.05718E11|   0| FY|     4|  23|
|320193|0000320193-19-000119|2019-09-30|ContractWithCusto...|us-gaap/2019|2018-09-30|USD|   5.966E9|   0| FY|     4|  20|
|320193|0000320193-19-000119|2019-09-30|ContractWithCusto...|us-gaap/2019|2019-09-30|USD|   5.522E9|   0| FY|     4|  20|
|320193|0000320193-19-000119|2019-09-30|              Assets|us-gaap/2019|2019-09-30|USD|3.38516E11|   0| FY|     4|  15|
|320193|0000320193-19-00

Checking the "ddate" column, we see entries that are in the past (compared to the "period" field  - which is the Balance Sheet Date, rounded to nearest month-end). This is normal, since the balancesheet also contains the data of the balance sheet from a year ago. However, in our case we are only interested in the data for the actual period. These are the entries where period and ddate have the same value.

In [ ]:
apple_bs_per_period = apple_vip_cols.where("period == ddate").orderBy(["cik","adsh","period","report","line"])

In [ ]:
apple_bs_per_period.show(32)

+------+--------------------+----------+--------------------+------------+----------+------+----------+----+---+------+----+
|   cik|                adsh|    period|                 tag|     version|     ddate|   uom|     value|qtrs| fp|report|line|
+------+--------------------+----------+--------------------+------------+----------+------+----------+----+---+------+----+
|320193|0000320193-19-000076|2019-06-30|CashAndCashEquiva...|us-gaap/2018|2019-06-30|   USD|  5.053E10|   0| Q3|     4|   3|
|320193|0000320193-19-000076|2019-06-30|MarketableSecurit...|us-gaap/2018|2019-06-30|   USD| 4.4084E10|   0| Q3|     4|   4|
|320193|0000320193-19-000076|2019-06-30|AccountsReceivabl...|us-gaap/2018|2019-06-30|   USD| 1.4148E10|   0| Q3|     4|   5|
|320193|0000320193-19-000076|2019-06-30|        InventoryNet|us-gaap/2018|2019-06-30|   USD|   3.355E9|   0| Q3|     4|   6|
|320193|0000320193-19-000076|2019-06-30|NontradeReceivabl...|us-gaap/2018|2019-06-30|   USD| 1.2326E10|   0| Q3|     4|   7|


Comparing the data above with the BalanceSheet in the appropriate report (https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019319000076/a10-qq320196292019.htm) we see that the data and entries match.

Finally, we pivot the data and we expect two rows in the data.

In [ ]:
apple_pivoted_df = apple_bs_per_period.select(["cik","adsh","period","ddate",'tag','value']) \
                                      .groupby(["cik","adsh","period","ddate"]) \
                                      .pivot("tag",['Assets','AssetsCurrent','OtherAssetsCurrent']).max('value')

In [ ]:
apple_pivoted_df.select(["cik","adsh","period",'ddate', 'Assets','AssetsCurrent','OtherAssetsCurrent']).show()

+------+--------------------+----------+----------+----------+-------------+------------------+
|   cik|                adsh|    period|     ddate|    Assets|AssetsCurrent|OtherAssetsCurrent|
+------+--------------------+----------+----------+----------+-------------+------------------+
|320193|0000320193-19-000119|2019-09-30|2019-09-30|3.38516E11|   1.62819E11|         1.2352E10|
|320193|0000320193-19-000076|2019-06-30|2019-06-30|3.22239E11|   1.34973E11|          1.053E10|
+------+--------------------+----------+----------+----------+-------------+------------------+



The result looks promising.

## Deciding which tags to pivot

In the analysis step we created a sorted list of the tags that are present in BalanceSheets. As was shown there, it doesn't make sense to pivot all 3400 tags. Instead, only a small subset appears often enough in reports to be useful. <br>
We stored the sorted list in the file "bs_tags.csv". No, we will load it and use the first 100 tags to define which values should be pivoted.

In [ ]:
bs_tags = pd.read_csv("./bs_tags.csv")['tag']

In [ ]:
relevant_tags = bs_tags[:100].tolist()

## Pivoting

### Pivot the testset

In [ ]:
df_test_bs_ready = df_tst.where("stmt = 'BS' and period == ddate").select(['cik','adsh','period','tag', 'ddate','value']).cache()

In [ ]:
df_test_bs_ready.count()

208895

In [ ]:
df_test_bs_ready.select('tag').distinct().count()

1368

In [ ]:
df_test_bs_pivot = df_test_bs_ready.groupby(["cik","adsh","period","ddate"]).pivot("tag",relevant_tags) \
                                   .max('value').cache()

In [ ]:
df_test_bs_pivot.count()

5639

In [ ]:
df_test_bs_pivot.write.parquet(tst_bs_folder)

### Pivot the whole dataset

In [ ]:
df_all_bs_ready = df_all.where("stmt = 'BS' and period == ddate").select(['cik','adsh','period','tag', 'ddate','value']).cache()

In [ ]:
df_all_bs_ready.count()

4720704

In [ ]:
df_all_bs_ready.select('tag').distinct().count()

2314

In [ ]:
df_all_bs_pivot = df_all_bs_ready.groupby(["cik","adsh","period","ddate"]).pivot("tag",relevant_tags) \
                                 .max('value').cache()

In [ ]:
df_all_bs_pivot.count()

131180

In order to have an easy way to have a look at the data with a texteditor, we convert it to a pandas Dataframe and store it as CSV. The resulting file size is now 54 MB. 

In [ ]:
df_all_bs_pivot.toPandas().to_csv("bs_data.csv",index=False,header=True)

But for further processing, we also store it as parquet, since this will keep that datatype information of the columns.

In [ ]:
shutil.rmtree(all_bs_folder,  ignore_errors=True)
df_all_bs_pivot.repartition(8,col("cik")).write.parquet(all_bs_folder)

## Stop the SparkContext

In [ ]:
spark.stop()